In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    getSharesIdx,
    getSharesIdxCI95,
    load_hh_raw,
    load_person_demographics,
    load_trip_assign_day,
    out_dir,
    prep_data_1d,
    prep_data_2d,
    tour_extract_allwk_dir,
    tour_extract_wkday_dir,
    trip_join_hh_person,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "02_AllTrips_Purpose.xlsx"

In [ ]:
hh = load_hh_raw(income=True, home_county=False)
person = load_person_demographics()

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
trip = trip_join_hh_person(
    load_trip_assign_day(
        tour_extract_wkday_dir / "trip-assign_day.csv", depart_hour=True
    ),
    hh,
    person,
).to_pandas()

In [ ]:
row = 0
sname = "Weekday"
d1_dict = df_to_excel_col_dict["dpurp"]
title = "All Trips by " + d1_dict["desc"]

tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)
title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)
tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]
title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for key in ["depart_hour", "raceeth", "hh_income_broad_cat", "pagey", "pgend"]:
    d2_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab = prep_data_2d(
            trip,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

        if wc == "count":
            tab2 = tab.copy()
        else:
            tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

        title = (
            "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

        title = (
            "Column Shares 95% CI by "
            + d2_dict["desc"]
            + " and "
            + d1_dict["desc"]
            + " "
            + wd
        )
        row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
## Process all week trip records
trip = trip_join_hh_person(
    load_trip_assign_day(tour_extract_allwk_dir / "trip-assign_day.csv"), hh, person
).to_pandas()

In [ ]:
row = 0
sname = "DOW"
d2_dict = df_to_excel_col_dict["day"]

for wc, wd in zip(wt_cols, wt_desc):
    title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    tab = prep_data_2d(
        trip,
        d1_dict["col"],
        d1_dict["vals"],
        d1_dict["labels"],
        d2_dict["col"],
        d2_dict["vals"],
        d2_dict["labels"],
        wc,
    )
    row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

    if wc == "count":
        tab2 = tab.copy()
    else:
        tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

    title = "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    title = (
        "Column Shares 95% CI by "
        + d2_dict["desc"]
        + " and "
        + d1_dict["desc"]
        + " "
        + wd
    )
    row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()